#### improvement plan in it
#### 1.replace weight,bias with nn module
#### 2. replace forward function
#### 3. use built in loss function
#### 4. use built in optimizer

In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [2]:
df = pd.read_csv("/content/breast_cancer_data.csv")

In [3]:
df.head(2)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN


In [4]:
df.drop(columns=['id','Unnamed: 32'],axis=1,inplace=True)

In [5]:
df.head(2)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902


In [6]:
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2,random_state=0)

In [7]:
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)


In [8]:
lb = LabelEncoder()
y_train_scaled = lb.fit_transform(y_train)
y_test_scaled = lb.transform(y_test)

#### convert numpy array into tensor

In [9]:
X_train_tensor = torch.from_numpy(X_train_scaled.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test_scaled.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train_scaled.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test_scaled.astype(np.float32))


In [10]:
X_train_tensor.dtype

torch.float32

In [11]:
X_train_tensor.shape

torch.Size([455, 30])

In [12]:
y_train_tensor.shape

torch.Size([455])

In [13]:
y_train_tensor.dtype

torch.float32

#### define model

In [25]:
class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features,1)
    self.sigmoid = nn.Sigmoid()

  def forward(self ,features):
    out= self.linear(features)
    out = self.sigmoid(out)
    return out

  # def loss_function(self,y_pred,y):
  #   # clamp prediction to avoid log(0)
  #   epsilon = 1e-7
  #   y_pred = torch.clamp(y_pred,epsilon,1-epsilon)

  #   # calculate loss
  #   loss = - (y_train_tensor * torch.log(y_pred) +(1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
  #   return loss

In [26]:
learning_rate = 0.1
epochs = 25


In [27]:
# using built in loss function

loss_function = nn.BCELoss()

#### training pipeline

In [29]:
model = MySimpleNN(X_train_tensor.shape[1])

# using built in optimizer

optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model(X_train_tensor)

  # calculate loss
  #loss = model.loss_function(y_pred,y_train_tensor)
  loss = loss_function(y_pred,y_train_tensor.view(-1,1))

  # clear gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # # parameter update
  # with torch.no_grad():
  #   model.linear.weight -= learning_rate * model.linear.weight.grad
  #   model.linear.bias -= learning_rate * model.linear.bias.grad
  optimizer.step()
  # # zero gardients
  # model.linear.weight.grad.zero_()
  # model.linear.bias.grad.zero_()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1} , loss: {loss.item()}')

Epoch: 1 , loss: 0.8957967758178711
Epoch: 2 , loss: 0.6324641704559326
Epoch: 3 , loss: 0.49823662638664246
Epoch: 4 , loss: 0.42237773537635803
Epoch: 5 , loss: 0.3734181821346283
Epoch: 6 , loss: 0.33882829546928406
Epoch: 7 , loss: 0.31283140182495117
Epoch: 8 , loss: 0.29240652918815613
Epoch: 9 , loss: 0.27581915259361267
Epoch: 10 , loss: 0.2620004117488861
Epoch: 11 , loss: 0.25025415420532227
Epoch: 12 , loss: 0.24010629951953888
Epoch: 13 , loss: 0.2312220335006714
Epoch: 14 , loss: 0.22335754334926605
Epoch: 15 , loss: 0.2163306027650833
Epoch: 16 , loss: 0.21000193059444427
Epoch: 17 , loss: 0.20426306128501892
Epoch: 18 , loss: 0.19902798533439636
Epoch: 19 , loss: 0.19422754645347595
Epoch: 20 , loss: 0.18980538845062256
Epoch: 21 , loss: 0.1857149302959442
Epoch: 22 , loss: 0.18191741406917572
Epoch: 23 , loss: 0.17838019132614136
Epoch: 24 , loss: 0.1750754714012146
Epoch: 25 , loss: 0.17197956144809723


In [30]:
model.linear.weight

Parameter containing:
tensor([[ 0.1668,  0.0263,  0.2337,  0.0724,  0.1274,  0.0385,  0.1831,  0.3764,
         -0.0059,  0.0975,  0.2638,  0.0854,  0.0974,  0.2216, -0.0703,  0.0607,
         -0.0364, -0.0645, -0.0737, -0.1610,  0.3668,  0.3122,  0.4404,  0.1360,
          0.0565,  0.0965,  0.3806,  0.4241,  0.1120, -0.0223]],
       requires_grad=True)

In [31]:
model.linear.bias

Parameter containing:
tensor([-0.0800], requires_grad=True)

#### evaluations

In [32]:
# Model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy : {accuracy.item()}')

Accuracy : 0.5123114585876465
